In [4]:
# -*- encoding: utf-8 -*-
# @Date    :   2025/01/06 11:36:22
# @Author  :   Hongtai Wang (ongtuai113@gmail.com)

import os
import xarray as xr

def integrate_nc_files(input_dir, output_file, lat_range, lon_range, var):
    """
    Integrates a specified variable and its associated coordinates (longitude, latitude, time) from all .nc files 
    in the specified directory within a given latitude and longitude range into one .nc file.

    Parameters:
        input_dir (str): The path to the directory containing .nc files.
        output_file (str): The path to save the integrated .nc file.
        lat_range (tuple): Latitude range as (min_lat, max_lat).
        lon_range (tuple): Longitude range as (min_lon, max_lon).
        var (str): The name of the variable to integrate.
    """
    datasets = []

    for file in os.listdir(input_dir):
        if file.endswith(".nc"):
            file_path = os.path.join(input_dir, file)
            try:
                # Open the .nc file
                ds = xr.open_dataset(file_path)
                
                # Subset by latitude and longitude range
                ds_filtered = ds.sel(
                    latitude=slice(lat_range[0], lat_range[1]),
                    longitude=slice(lon_range[0], lon_range[1])
                )
                
                # Ensure the required variable is present
                vars_to_select = var + ['longitude', 'latitude', 'time']
                ds_selected = ds_filtered[vars_to_select]
                datasets.append(ds_selected)
            except KeyError as e:
                print(f"Warning: File {file} is missing the variable or coordinates: {e}")
            except Exception as e:
                print(f"Error processing file {file}: {e}")

    if not datasets:
        print("No valid .nc files found in the specified range. Exiting.")
        return

    # Merge all datasets
    try:
        combined_ds = xr.concat(datasets, dim="time")
    except Exception as e:
        print(f"Error during concatenation: {e}")
        return

    # Save to output .nc file
    try:
        combined_ds.to_netcdf(output_file)
        print(f"Integrated .nc file saved to {output_file}")
    except Exception as e:
        print(f"Error saving the .nc file: {e}")

# Example usage:
input_dir = "D:/data/CMEMS/MY/L4/SSH"
output_file = "D:/data/CMEMS/SCS/SSH_SLA.nc"
lat_range = (0, 25)  # Latitude range: 0° to 25°N
lon_range = (105, 130)  # Longitude range: 105°E to 130°E

# List all variables in the first .nc file
for file in os.listdir(input_dir):
    if file.endswith(".nc"):
        file_path = os.path.join(input_dir, file)
        try:
            ds = xr.open_dataset(file_path)
            print(f"Variables in {file}: {list(ds.variables.keys())}")
            break
        except Exception as e:
            print(f"Error opening file {file}: {e}")

var = ["adt", "ugos", "vgos","sla","ugosa","vgosa"] # Variable to integrate
integrate_nc_files(input_dir, output_file, lat_range, lon_range, var)


Variables in dt_global_allsat_phy_l4_20170101_20241017.nc: ['crs', 'time', 'latitude', 'lat_bnds', 'longitude', 'lon_bnds', 'nv', 'sla', 'err_sla', 'ugosa', 'err_ugosa', 'vgosa', 'err_vgosa', 'adt', 'ugos', 'vgos', 'flag_ice', 'tpa_correction']
Integrated .nc file saved to D:/data/CMEMS/SCS/SSH_SLA.nc
